# Evolution de l'émission du centre galactique et de l'émission diffuse

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.convolution import Tophat2DKernel
from regions import CircleSkyRegion, RectangleSkyRegion

from gammapy.detect import compute_lima_on_off_image
from gammapy.data import DataStore
from gammapy.irf import make_mean_psf
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.cube import (
    MapDatasetMaker,
    PSFKernel,
    MapDataset,
    RingBackgroundMaker,
    SafeMaskMaker,
    #RingBackgroundEstimator,
)
from gammapy.modeling.models import (
    SkyModel,
    BackgroundModel,
    PowerLawSpectralModel,
    PowerLaw2SpectralModel,
    PointSpatialModel,
    ExpCutoffPowerLawSpectralModel,
    SkyDiffuseCube,
    TemplateSpatialModel
)
from gammapy.modeling import Fit
from astropy.time import Time

src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

import gammapy
gammapy.__version__

'0.15'

Emprunt d'une classe implémentée en 0.16

In [2]:
##FoV background estimation
import logging
from gammapy.maps import Map
from gammapy.modeling import Fit, Datasets


class FoVBackgroundMaker:
    """Normalize template background on the whole field-of-view.

    The dataset background model can be simply scaled (method="scale") or fitted (method="fit")
    on the dataset counts.

    The normalization is performed outside the exclusion mask that is passed on init.

    If a SkyModel is set on the input dataset and method is 'fit', its are frozen during
    the fov normalization fit.

    Parameters
    ----------
    method : str in ['fit', 'scale']
        the normalization method to be applied. Default 'scale'.
    exclusion_mask : `~gammapy.maps.WcsNDMap`
        Exclusion mask
    """

    def __init__(self, method="scale", exclusion_mask=None):
        if method in ["fit", "scale"]:
            self.method = method
        else:
            raise ValueError(f"Incorrect method for FoVBackgroundMaker: {method}.")
        self.exclusion_mask = exclusion_mask


    def run(self, dataset):
        """Run FoV background maker.

        Fit the background model norm

        Parameters
        ----------
        dataset : `~gammapy.cube.fit.MapDataset`
            Input map dataset.

        """
        mask_fit = dataset.mask_fit
        dataset.mask_fit = self._reproject_exclusion_mask(dataset)

        if self.method is "fit":
            self._fit_bkg(dataset)
        else:
            self._scale_bkg(dataset)

        dataset.mask_fit = mask_fit
        return dataset


    def _reproject_exclusion_mask(self, dataset):
        """Reproject the exclusion on the dataset geometry"""
        mask_map = Map.from_geom(dataset.counts.geom)
        if self.exclusion_mask is not None:
            coords = dataset.counts.geom.get_coord()
            vals = self.exclusion_mask.get_by_coord(coords)
            mask_map.data += vals

        return mask_map.data.astype("bool")

    def _fit_bkg(self, dataset):
        """Fit the FoV background model on the dataset counts data"""

        # freeze all model components not related to background model
        datasets = Datasets([dataset])

        parameters_frozen = []
        for par in datasets.parameters:
            parameters_frozen.append(par.frozen)
            if par not in dataset.background_model.parameters:
                par.frozen = True

        #!!!AL: relax titlt : BE CARREFULL !!!
        dataset.background_model.tilt.frozen=False
        
        fit = Fit(datasets)
        fit_result = fit.run()
        if fit_result.success is False:
            print("FoVBackgroundMaker failed. No fit convergence")
            

        # Unfreeze parameters
        for i, par in enumerate(datasets.parameters):
            par.frozen = parameters_frozen[i]

    def _scale_bkg(self, dataset):
        """Fit the FoV background model on the dataset counts data"""
        mask = dataset.mask
        count_tot = dataset.counts.data[mask].sum()
        bkg_tot = dataset.background_model.map.data[mask].sum()

        if count_tot <= 0.0:
            print("FoVBackgroundMaker failed. No counts found outside exclusion mask")
        elif bkg_tot <= 0.0:
            print("FoVBackgroundMaker failed. No positive background found outside exclusion mask")
        else:
            scale = count_tot / bkg_tot
            dataset.background_model.norm.value = scale
            #print("bkg scale = ",scale)

## Fabrication des mapdatasets

Define which data to use and print some information

In [3]:
data_store_hess1 = DataStore.from_dir("$GAMMAPY_DATA/hap-hd_Prod05/hess1/std_ImPACT_fullEnclosure")
data_store_hess1u = DataStore.from_dir("$GAMMAPY_DATA/hap-hd_Prod05/hess1u/std_ImPACT_fullEnclosure")
data_store_hess2 = DataStore.from_dir("$GAMMAPY_DATA/hap-hd_Prod05/hess2/std_ImPACT_fullEnclosure")

In [4]:
from astropy.coordinates import Angle


selection = dict(type='sky_circle', frame='galactic',
                 lon=Angle(0, 'deg'),
                 lat=Angle(0, 'deg'),
                 radius=Angle(2, 'deg'),
                 border=Angle(0, 'deg'))

obs_table1 = data_store_hess1.obs_table.select_observations(selection)
obs_table1u = data_store_hess1u.obs_table.select_observations(selection)
obs_table2 = data_store_hess2.obs_table.select_observations(selection)

In [38]:
obs_table2.time_start

<Time object: scale='tt' format='mjd' value=[56452.84258315 56452.86352065 56452.884435   56452.90534935
 56452.92637944 56452.94730537 56452.96833546 56452.97964333
 56453.00066185 56456.88909935 56456.91826602 56456.93928454
 56456.96350907 56456.98452759 56457.00554611 56457.02656463
 56457.04075444 56471.87213176 56472.87281463 56473.87053454
 56473.89155306 56473.91258315 56473.93360167 56473.95462019
 56474.85113639 56474.89372898 56474.93651833 56474.97874056
 56476.85180769 56478.92235167 56478.96540722 56479.85420352
 56479.8847475  56479.90576602 56479.92540722 56479.94642574
 56479.96735167 56479.98837019 56480.0093887  56480.03040722
 56481.85418037 56481.90301139 56481.9169813  56481.93929611
 56481.96031463 56482.83173824 56482.88059241 56485.92038407
 56508.89847435 56509.81369426 56509.90893731 56509.92986324
 56510.78109009 56510.84712019 56510.88924981 56510.93243269
 56511.80146046 56511.82238639 56511.84353222 56511.8607313
 56511.891935   56511.91593963 56512.80900

In [5]:
t2004  = dict(type='time_box', time_range= Time(['2004-01-01T00:00:00', '2004-12-31T23:59:59']))
t2005  = dict(type='time_box', time_range= Time(['2005-01-01T00:00:00', '2005-12-31T23:59:59']))
t2006  = dict(type='time_box', time_range= Time(['2006-01-01T00:00:00', '2006-12-31T23:59:59']))
t2007  = dict(type='time_box', time_range= Time(['2007-01-01T00:00:00', '2007-12-31T23:59:59']))
t2008  = dict(type='time_box', time_range= Time(['2008-01-01T00:00:00', '2008-12-31T23:59:59']))
t2009  = dict(type='time_box', time_range= Time(['2009-01-01T00:00:00', '2009-12-31T23:59:59']))
t2010  = dict(type='time_box', time_range= Time(['2010-01-01T00:00:00', '2010-12-31T23:59:59']))
t2011  = dict(type='time_box', time_range= Time(['2011-01-01T00:00:00', '2011-12-31T23:59:59']))
t2012  = dict(type='time_box', time_range= Time(['2012-01-01T00:00:00', '2012-12-31T23:59:59']))
t2013  = dict(type='time_box', time_range= Time(['2013-01-01T00:00:00', '2013-12-31T23:59:59']))
t2014  = dict(type='time_box', time_range= Time(['2014-01-01T00:00:00', '2014-12-31T23:59:59']))
t2015  = dict(type='time_box', time_range= Time(['2015-01-01T00:00:00', '2015-12-31T23:59:59']))
t2016  = dict(type='time_box', time_range= Time(['2016-01-01T00:00:00', '2016-12-31T23:59:59']))
t2017  = dict(type='time_box', time_range= Time(['2017-01-01T00:00:00', '2017-12-31T23:59:59']))
t2018  = dict(type='time_box', time_range= Time(['2018-01-01T00:00:00', '2018-12-31T23:59:59']))
t2019  = dict(type='time_box', time_range= Time(['2019-01-01T00:00:00', '2019-12-31T23:59:59']))

Sélection par année et tri des observations (on retire celles qui n'ont pas toutes les IRF)

In [6]:
year_intervals = { 2004 : t2004, 2005 : t2005, 2006 : t2006, 2007 : t2007,
                      2008 : t2008, 2009 : t2009, 2010 : t2010, 2011 : t2011,
                      2012 : t2012, 2013 : t2013, 2014 : t2014, 2015 : t2015,
                      2016 : t2016, 2017 : t2017, 2018 : t2018, 2019 : t2019}

yearly_obs1 = dict()

for year in range(2004,2020) :
    
    obs_table_year = obs_table1.select_observations(year_intervals[year])
    ids = obs_table_year["OBS_ID"].tolist()
    observations_year = data_store_hess1.get_observations(ids, skip_missing=True)
    
    for obs in observations_year:
        try:
            obs.aeff
            obs.edisp
            obs.psf
            obs.bkg
        except:
            ids.remove(obs.obs_id)
            print("Observation retirée : " + str(obs.obs_id))
            
    observations_year = data_store_hess1.get_observations(ids, skip_missing=True)
    yearly_obs1[year] = observations_year

Observation retirée : 20191
Observation retirée : 27162
Observation retirée : 27204
Observation retirée : 27329
Observation retirée : 27578
Observation retirée : 27607
Observation retirée : 27638


In [66]:
yearly_obs1u = dict()

for year in range(2004,2020) :
    
    obs_table_year = obs_table1u.select_observations(year_intervals[year])
    ids = obs_table_year["OBS_ID"].tolist()
    observations_year = data_store_hess1u.get_observations(ids, skip_missing=True)
    
    for obs in observations_year:
        try:
            obs.aeff
            obs.edisp
            obs.psf
            obs.bkg
        except:
            ids.remove(obs.obs_id)
            print("Observation retirée : " + str(obs.obs_id))
            
    observations_year = data_store_hess1u.get_observations(ids, skip_missing=True)
    yearly_obs1u[year] = observations_year

Observation retirée : 146924
Observation retirée : 146965
Observation retirée : 147004
Observation retirée : 147007
Observation retirée : 147008
Observation retirée : 147009
Observation retirée : 147010
Observation retirée : 147011
Observation retirée : 147012
Observation retirée : 147055
Observation retirée : 147056
Observation retirée : 147077
Observation retirée : 147121
Observation retirée : 147124
Observation retirée : 153153


In [7]:
yearly_obs2 = dict()

for year in range(2004,2020) :
    
    obs_table_year = obs_table2.select_observations(year_intervals[year])
    ids = obs_table_year["OBS_ID"].tolist()
    observations_year = data_store_hess2.get_observations(ids, skip_missing=True)
    
    for obs in observations_year:
        try:
            obs.aeff
            obs.edisp
            obs.psf
            obs.bkg
        except:
            ids.remove(obs.obs_id)
            print("Observation retirée : " + str(obs.obs_id))
            
    observations_year = data_store_hess2.get_observations(ids, skip_missing=True)
    yearly_obs2[year] = observations_year

In [61]:
obs_table1.select_observations(year_intervals[2004])

ALT_PNT,AZ_PNT,DATE-END,DATE-OBS,DEADC,DEC_OBJ,DEC_PNT,EVENT_COUNT,EVENT_DEC_MEDIAN,EVENT_ENERGY_MEDIAN,EVENT_RA_MEDIAN,EVENT_TIME_MAX,EVENT_TIME_MIN,GLAT_PNT,GLON_PNT,LIVETIME,MUONCORR,MUONEFF,N_TELS,OBJECT,OBS_ID,ONTIME,QUALITY,RA_OBJ,RA_PNT,TELLIST,TIME-END,TIME-OBS,TSTART,TSTOP,ZEN_PNT,BKG_SCALE
float32,float32,bytes10,bytes10,float32,float32,float32,int64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,int64,bytes27,int64,float32,int64,float32,float32,bytes7,bytes12,bytes12,float64,float64,float32,float32


## Création de la géométrie

In [8]:
emin, emax = [0.5, 100] * u.TeV

energy_axis = MapAxis.from_bounds(
    emin.value, emax.value, 20, unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    coordsys="GAL",
    proj="CAR",
    axes=[energy_axis],
)

energy_axis_true = MapAxis.from_bounds(
    0.3, 200, 30, unit="TeV", name="energy", interp="log"
)

# peut être passer à plus que 20 bins en énergie vraie ?

In [9]:
mapdataset_dict = {}

for k in range (2004,2020):
    name = "map" + str(k)
    mapdataset_dict[k] = MapDataset.create(
    geom=geom, energy_axis_true=energy_axis_true, name=name)

A noter que nos datasets ont un axe d'énergie vraie, pas biné très finement cela dit...

## Fabrication des mapdatasets

In [10]:
%%time

exclusion_region = RectangleSkyRegion(src_pos, 3*u.deg, 1*u.deg)
exclusion_mask = geom.region_mask([exclusion_region], inside=False)
exclusion_mask = Map.from_geom(geom, data=exclusion_mask)

for year in range(2004,2020):
    
    offset_max = 2.0 * u.deg
    maker = MapDatasetMaker()
    maker_safe_mask = SafeMaskMaker(methods=["offset-max", "bkg-peak"], offset_max=offset_max)
    maker_bkg = FoVBackgroundMaker("scale", exclusion_mask)
    
    # au lieu de "scale" prendre "fit" en donnant au dataset un masque sur les bin en énergie (prendre à partir de 1 TeV par ex, même si c'est déjà haut 500 GeV)
    # ou alors on met "bkg-peak" pour le safemaskmaker (le premier bin en énergie est retiré)
    
    # refaire des mapdataset en excluant le premier bin pour voir si ça change
    
    spectrum = PowerLaw2SpectralModel(index=2.3)

    for obs in yearly_obs1[year]:
        # First a cutout of the target map is produced
        cutout = mapdataset_dict[year].cutout(obs.pointing_radec, width=2 * offset_max)

        # A MapDataset is filled in this cutout geometry
        dataset = maker.run(cutout, obs)
        
        # The data quality cut is applied
        dataset = maker_safe_mask.run(dataset, obs)
        
        # stocker la distributions des seuils par années (utiliser np.where(mask.data[:, 250,250]))
        
        dataset = maker_bkg.run(dataset)
        
        # The resulting dataset cutout is stacked onto the final one
        mapdataset_dict[year].stack(dataset)
        
    for obs in yearly_obs1u[year]:
        # First a cutout of the target map is produced
        cutout = mapdataset_dict[year].cutout(obs.pointing_radec, width=2 * offset_max)

        # A MapDataset is filled in this cutout geometry
        dataset = maker.run(cutout, obs)
        
        # The data quality cut is applied
        dataset = maker_safe_mask.run(dataset, obs)
        
        # stocker la distributions des seuils par années (utiliser np.where(mask.data[:, 250,250]))
        
        dataset = maker_bkg.run(dataset)
        
        # The resulting dataset cutout is stacked onto the final one
        mapdataset_dict[year].stack(dataset)
        
    for obs in yearly_obs2[year]:
        # First a cutout of the target map is produced
        cutout = mapdataset_dict[year].cutout(obs.pointing_radec, width=2 * offset_max)

        # A MapDataset is filled in this cutout geometry
        dataset = maker.run(cutout, obs)
        
        # The data quality cut is applied
        dataset = maker_safe_mask.run(dataset, obs)
        
        # stocker la distributions des seuils par années (utiliser np.where(mask.data[:, 250,250]))
        
        dataset = maker_bkg.run(dataset)
        
        # The resulting dataset cutout is stacked onto the final one
        mapdataset_dict[year].stack(dataset)
        
    #mask = dataset.mask_safe.copy()
    #mask.data[0:3,:,:] = False
    #dataset.mask_fit = mask
    
    # essayer de se placer systématiquement au dessus du seuil

NameError: name 'yearly_obs1u' is not defined

In [53]:
safemask = mapdataset_dict[2004].mask_safe
expo = mapdataset_dict[2004].exposure

In [54]:
reduce = safemask.reduce_over_axes(func=np.logical_or)
safemask.data[:, 250 ,250]

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

Lors du stack on n'est pas surs de comment les différents seuils sont pris en compte.
Regardez les différentes énergies seuils, voir la distribution et enlever ceux qui sont trop haut. Ou bien juste partir beaucoup plus haut que le seuil donné par bkg-peak.

Eventuellement participer à l'implémentation de

In [55]:
# Pour information, la fonction associée à 'bkg-peak', rien à changer a priori, c'est simple et clair. ça fait ce que c'est censé faire, 
# à la limite on peut penser à d'autres méthodes pour couper en énergie

def make_mask_energy_bkg_peak(dataset):
        """Make safe energy mask based on the binned background.

        The energy threshold is defined as the upper edge of the energy
        bin with the highest predicted background rate. This method is motivated
        by its use in the HESS DL3 validation paper: https://arxiv.org/pdf/1910.08088.pdf

        Parameters
        ----------
        dataset : `~gammapy.modeling.Dataset`
            Dataset to compute mask for.

        Returns
        -------
        mask_safe : `~numpy.ndarray`
            Safe data range mask.
        """

        if isinstance(dataset, (MapDataset, MapDatasetOnOff)):
            background_spectrum = dataset.background_model.map.get_spectrum()
            counts = dataset.counts.geom
        else:
            background_spectrum = dataset.background
            counts = dataset.counts

        idx = np.argmax(background_spectrum.data)         # trouve l'indice du max du spectre du bkg
        e_min = background_spectrum.energy.edges[idx + 1] # identifie l'emin comme l'énergie suivante du max
        return counts.energy_mask(emin=e_min)

In [ ]:
# Pour information, la méthode 'stack' de la classe MapDataset, on a un mapdataset (self) auquel on en rajoute un autre (other)

def stack(self, other):
        """Stack another dataset in place.

        Parameters
        ----------
        other: `~gammapy.cube.MapDataset`
            Map dataset to be stacked with this one.
        """

        if self.counts and other.counts:
            self.counts *= self.mask_safe
            self.counts.stack(other.counts, weights=other.mask_safe)

        if self.exposure and other.exposure:                                     # ICI
            mask_image = self.mask_safe.reduce_over_axes(func=np.logical_or)
            self.exposure *= mask_image.data
            
            # TODO: apply energy dependent mask to exposure. Does this require
            #  a mask_safe in true energy?
            
            mask_image_other = other.mask_safe.reduce_over_axes(func=np.logical_or)
            self.exposure.stack(other.exposure, weights=mask_image_other)
            
            # on applique reduce_over_axes aux deux safe_mask, donc on les réduit sur les axes non-spatiaux 
            # (mais on prend le meilleur masque si les énergies sont masquées différemment)
            # donc on ne prend pas en compte le masquage éventuel en énergie lorsqu'on modifie l'exposure, 
            # dans : self.exposure *= mask_image.data (elle est multipliée d'un bloc)
            
            # derrière il semble avoir un souci entre énergie vraie et énergie recombinée
            
            # On finit avec la fonction stack de WcsNDMap, 
            # avec des weights qui RAJOUTENT le nouveau masque 
            # (s'il est plus grand, on supprime de nouveaux bins, s'il est plus petit, les bins effacés le restent), logique avec le concept de 'stack'
            # la question est de savoir si appliquer un masque energie dépendant sur la carte d'exposition nécessite un mask safe en énergie vraie
            # oui car l'exposition a des bins en énergie vraie et pas énergie recombinée
            # mais il suffirait de "faire la conversion"
            
        if self.background_model and other.background_model:    
            bkg = self.background_model.evaluate()
            bkg *= self.mask_safe
            other_bkg = other.background_model.evaluate()
            bkg.stack(other_bkg, weights=other.mask_safe)

            self.background_model = BackgroundModel(
                bkg, name=self.background_model.name
            )

        if self.mask_safe is not None and other.mask_safe is not None:            # ICI en fait non ya rien à changer ici
            self.mask_safe.stack(other.mask_safe)
            
            # juste une fonction 'stack' celle de WcsNDMap

        if self.psf and other.psf:
            if isinstance(self.psf, PSFMap) and isinstance(other.psf, PSFMap):
                mask_irf = self._mask_safe_irf(self.psf.psf_map, mask_image)
                self.psf.psf_map *= mask_irf.data
                self.psf.exposure_map *= mask_irf.data

                mask_image_other = other.mask_safe.reduce_over_axes(func=np.logical_or)
                mask_irf_other = self._mask_safe_irf(
                    other.psf.psf_map, mask_image_other
                )
                self.psf.stack(other.psf, weights=mask_irf_other)
            else:
                raise ValueError("Stacking of PSF kernels not supported")

        if self.edisp and other.edisp:
            if isinstance(self.edisp, EDispMap) and isinstance(other.edisp, EDispMap):
                mask_irf = self._mask_safe_irf(self.edisp.edisp_map, mask_image)
                self.edisp.edisp_map *= mask_irf.data
                self.edisp.exposure_map *= mask_irf.data

                mask_image_other = other.mask_safe.reduce_over_axes(func=np.logical_or)
                mask_irf_other = self._mask_safe_irf(
                    other.edisp.edisp_map, mask_image_other
                )
                self.edisp.stack(other.edisp, weights=mask_irf_other)
            else:
                raise ValueError("Stacking of edisp kernels not supported")

        if self.gti and other.gti:
            self.gti = self.gti.stack(other.gti).union()

In [ ]:
# Au sujet de l'énergie vraie : elle n'intervient que dans la fonction create où elle est donnée, et permet de créer les cubes pour l'exposition, la psf et l'edisp

geom_image = geom.to_image() # on prend la geom 3d on a passe en 2d pour la repasser en 3d avec l'axe d'énergie vraie
geom_exposure = geom_image.to_cube([energy_axis_true])
geom_irf = geom_image.to_binsz(binsz=binsz_irf) # ça on y touche pas apparemment
geom_psf = geom_irf.to_cube([rad_axis, energy_axis_true])
geom_edisp = geom_irf.to_cube([migra_axis, energy_axis_true])

In [56]:
# Conversion d'un masque en énergie recombinée en un masque en énergie vraie
# sachant qu'a priori on veut pas masquer d'emblée toutes les énergies vraies en dessous du seuil en énergie recombinée
# ça risque d'être lourd
# on peut faire une execption si on a le même nombre de bins pour les deux

def conv_mask_true(mask, erec, etrue):
    # mask est une carte WcsNDMap de booléens
    # erec et etrue sont des energy axis avec des edges et nbins a priori différents
    
    mask_true = mask.copy()
    
    # besoin d'une fonction qui pour un pixel regarde le dernièr bin en énergie rec masqué, et associe une liste de bin en énergie vraie masquée adéquatement

In [ ]:
# Pour info, méthode stack de la classe WcsNDMap

def stack(self, other, weights=None):
        """Stack cutout into map.

        Parameters
        ----------
        other : `WcsNDMap`
            Other map to stack
        weights : `WcsNDMap`
            Array to be used as weights.
        """
        if self.geom == other.geom:
            parent_slices, cutout_slices = None, None
        elif self.geom.is_aligned(other.geom):
            slices = other.geom.cutout_info["parent-slices"]
            parent_slices = Ellipsis, slices[0], slices[1]

            slices = other.geom.cutout_info["cutout-slices"]
            cutout_slices = Ellipsis, slices[0], slices[1]
        else:
            raise ValueError(
                "Can only stack equivalent maps or cutout of the same map."
            )

        data = other.data[cutout_slices]

        if weights is not None:
            data = data * weights.data

        self.data[parent_slices] += data

In [43]:
mapdataset_dict[2006].mask_safe.data[:,250,250]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [42]:
print(energy_axis.edges)

[  0.7          0.89710471   1.14970981   1.47344298   1.88833234
   2.42004549   3.10147745   3.97478577   5.09399863   6.5283574
   8.36660027  10.72245218  13.74166054  17.61101204  22.56988842
  28.92507609  37.06974581  47.50777665  60.88492901  78.02879533
 100.        ] TeV


In [ ]:
# Pour masquer plus de bins que ce que fait le mask safe

mask = dataset.mask_safe.copy()
mask.data[0:3,:,:]=False
dataset.mask_fit = mask

Sauvegarde des mapdatasets

In [83]:
from pathlib import Path

#path = Path("../../../gamapy_data/mapdataset_hess/mapsdataset_hap-hd_zeta")
#path.mkdir(exist_ok=True)

for year in range(2004,2020):
    
    filename = "../../../gammapy_data/mapdataset_hess/mapsdataset_hap-hd_ImPACT/mapdataset" +str(year)+".fits.gz"
    mapdataset_dict[year].write(filename, overwrite=True)